In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/car-price-prediction/CarPrice_Assignment.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.drop(['car_ID','CarName'], axis='columns', inplace=True)
df.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.heatmap(df.isnull(),yticklabels=False,cbar=False)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True,cmap="YlGnBu")

In [ ]:
df_categorical=df.select_dtypes(include=['object']).copy()
df_categorical.head(3)

In [ ]:
df_categorical.columns

In [ ]:
for i in df_categorical.columns:
    print(df_categorical[i].unique())

In [ ]:
for i in df_categorical.columns:
    print(df_categorical[i].value_counts())

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df_categorical['fueltype']=encoder.fit_transform(df_categorical['fueltype'])
df_categorical['aspiration']=encoder.fit_transform(df_categorical['aspiration'])
df_categorical['doornumber']=encoder.fit_transform(df_categorical['doornumber'])
df_categorical['enginelocation']=encoder.fit_transform(df_categorical['enginelocation'])
df_categorical['enginetype']=encoder.fit_transform(df_categorical['enginetype'])
df_categorical['cylindernumber']=encoder.fit_transform(df_categorical['cylindernumber'])
df_categorical['fuelsystem']=encoder.fit_transform(df_categorical['fuelsystem'])
df_categorical['carbody']=encoder.fit_transform(df_categorical['carbody'])
df_categorical['drivewheel']=encoder.fit_transform(df_categorical['drivewheel'])

In [ ]:
df_categorical.head()

In [ ]:
df_numeric=df.select_dtypes(exclude=['object']).copy()
df_numeric.head(3)

In [ ]:
for i in df_numeric.columns:
    sns.boxplot(df_numeric[i])
    plt.show()

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
print((df_numeric < (Q1 - 1.5 * IQR)) |(df_numeric > (Q3 + 1.5 * IQR)))

In [ ]:
df_numeric.shape

In [ ]:
#removing all the outlier from the dataframe
df_numeric_remove_outlier = df_numeric[~((df_numeric < (Q1 - 1.5 * IQR)) |(df_numeric > (Q3 + 1.5 * IQR))).any(axis=1)]
df_numeric_remove_outlier.shape

In [ ]:
df_new = pd.concat([df_numeric_remove_outlier, df_categorical], axis=1, join="inner")

In [ ]:
df_new

In [ ]:
df_new.info()

In [ ]:
df_new.dropna(inplace=True)

In [ ]:
df_new.columns

In [ ]:
x=df_new.drop(['price'], axis='columns')
y=df_new['price']

In [ ]:
x.head(5)

In [ ]:
x.drop(['symboling'], axis='columns', inplace=True)

In [ ]:
y[:5]

In [ ]:
y=y.astype('int')

In [ ]:
y[:5]

In [ ]:
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bestfeatures=SelectKBest(score_func=chi2, k=10)
fit=bestfeatures.fit(x,y)
dfscores=pd.DataFrame(fit.scores_)
dfcolumns=pd.DataFrame(x.columns)
featureScores=pd.concat([dfcolumns,dfscores], axis=1)
featureScores.columns=['Specs','Scores']
featureScores.nlargest(n=15,columns='Scores')

In [ ]:
df_features=df_new[['curbweight','peakrpm','horsepower','enginesize','fuelsystem','citympg','highwaympg','aspiration','carlength','doornumber','enginetype','carbody','cylindernumber','wheelbase','drivewheel']]

In [ ]:
df_features.head()

In [ ]:
from sklearn.preprocessing import Normalizer
scaler=Normalizer()
x_scaled=scaler.fit_transform(df_features)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_scaled,y,test_size=0.25)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
lr.score(X_test,y_test)

In [ ]:
y_pred=lr.predict(X_test)

In [ ]:
y_test[:5]

In [ ]:
y_pred[:5]

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
c = [i for i in range(len(y_pred))]
fig = plt.figure()
plt.plot(c,y_test-y_pred, color="blue", linewidth=2.5, linestyle="-")
fig.suptitle('Error Terms using Linear Regression', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                      # X-label
plt.ylabel('ytest-ypred', fontsize=16)                # Y-label
plt.show()

In [ ]:

from sklearn.tree import DecisionTreeRegressor  
  

regressor = DecisionTreeRegressor(criterion='mse',random_state = 0)  
  

regressor.fit(X_train,y_train) 

In [ ]:
regressor.score(X_test,y_test)

In [ ]:
y_test[:5]

In [ ]:
y_pred_DTR=regressor.predict(X_test)
y_pred_DTR[:5]

In [ ]:
c = [i for i in range(len(y_pred))]
fig = plt.figure()
plt.plot(c,y_test-y_pred_DTR, color="red", linewidth=2.5, linestyle="-")
fig.suptitle('Error Terms using Decision Tree Regression', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                      # X-label
plt.ylabel('ytest-ypred', fontsize=16)                # Y-label
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
  

regressor_RF = RandomForestRegressor(criterion='mse',n_estimators = 100, random_state = 0) 
  

regressor_RF.fit(X_train,y_train)

In [ ]:
regressor_RF.score(X_test,y_test)

In [ ]:
y_test[:5]

In [ ]:
y_pred_RF=regressor.predict(X_test)
y_pred_RF[:5]

In [ ]:
c = [i for i in range(len(y_pred))]
fig = plt.figure()
plt.plot(c,y_test-y_pred_RF, color="green", linewidth=2.5, linestyle="-")
fig.suptitle('Error Terms using Random Forest Regression', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                      # X-label
plt.ylabel('ytest-ypred', fontsize=16)                # Y-label
plt.show()